# 수제버거 추천시스템

## 1. Data Crawling

In [ ]:
# 라이브러리 호출
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
import pandas as pd


In [ ]:

# 크롤링 함수
def review_crawling(id):    
    driver = webdriver.Chrome()
    df = pd.DataFrame(columns=['restaurant', 'username', 'userid_r','userid_p','userid_f', 'score',
                           'method', 'text', 'keyword','visit_date', 'visit_time'])
    url = 'https://m.place.naver.com/restaurant/'+str(id)+'/review/visitor'
    driver.get(url)
    time.sleep(3)

    restaurant = driver.find_element(By.CLASS_NAME, 'GHAhO').text
    while True:
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            button = driver.find_element(By.CLASS_NAME, 'fvwqf')
            button.send_keys(Keys.ENTER)
        except:
            break

    
    li = driver.find_elements(By.CLASS_NAME, 'owAeM')

    for i in li:
        name = i.find_element(By.CLASS_NAME, 'P9EZi').text # 고객 이름
        id_list = i.find_elements(By.CLASS_NAME, 'RNn6x') # 추가 식별자
        id_r = id_list[0].text.split(' ')[-1] # 리뷰
        try:
            id_p = id_list[1].text.split(' ')[-1] # 사진
        except:
            id_p = 0
        try:
            id_f = id_list[2].text.split(' ')[-1] # 팔로워
        except:
            id_f = 0
        
        methods = i.find_elements(By.CLASS_NAME, 'xalSr') # 이용 방법
        method = []
        for j in methods:
            method.append(j.text)

        try:
            i.find_element(By.CLASS_NAME, 'sIv5s.WPk67').send_keys(Keys.ENTER)
        except:
            try:
                i.find_element(By.CLASS_NAME, 'xHaT3').send_keys(Keys.ENTER)
            except:
                pass
            
        review_text = i.find_element(By.CLASS_NAME, 'zPfVt').text # 리뷰 내용
        
        emotions = i.find_elements(By.CLASS_NAME, 'sIv5s') # 키워드
        emotion = []
        for j in emotions:
            emotion.append(j.text)

        visit = i.find_elements(By.CLASS_NAME, 'CKUdu')
        visitdate = visit[0].text.split('\n')[-1]
        visittime = visit[1].text[0]

        try:
            score = i.find_element(By.CLASS_NAME, 'FMf0T').text.split('\n')[1]
        except:
            score = 'NaN'

        new_data = [restaurant, name, id_r, id_p, id_f, score, method, review_text, emotion, visitdate, visittime]
        df = pd.concat([df, pd.DataFrame([new_data], columns=df.columns)], ignore_index=True)

    driver.close()

    return df

# id 리스트 가져오기
id_df = pd.read_csv('id식별자.csv')

id_list = id_df[id_df['person']=='최주희']['id'].tolist() # 본인 이름 기입하기


# 데이터프레임 생성

final_df = pd.DataFrame(columns=['restaurant', 'username', 'userid_r','userid_p','userid_f','score',

                          'method', 'text', 'keyword','visit_date', 'visit_time'])


# 데이터 크롤링
for num in id_list:
    cache = review_crawling(num)
    final_df = pd.concat([final_df, cache])

#저장
final_df.to_csv('review_crawling(이름).csv') # 본인 이름 기입하기

## 2. Data Processing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

### 프리미엄 버거

In [ ]:
df =  pd.read_csv('/content/drive/MyDrive/24-2 KUBIG reco/burger_data/menu_crawling.csv')

In [ ]:
df.head()

,Unnamed: 0,ids,name,rank,visitor,blog,menu,explain,price,jibun_address
0,0,37282209,베이킹 도우,4.45,205,212,마르게리따,NaN,"16,500원",원서동 156
1,1,37282209,베이킹 도우,4.45,205,212,프로슈토 루꼴라,NaN,"24,500원",원서동 156
2,2,37282209,베이킹 도우,4.45,205,212,초리조,NaN,"18,500원",원서동 156
3,3,37282209,베이킹 도우,4.45,205,212,루꼴라,NaN,"19,500원",원서동 156
4,4,37282209,베이킹 도우,4.45,205,212,필스너 우르켈 생맥주,NaN,"8,000원",원서동 156


#### 1. 데이터 전처리

1. 데이터 변수명 수정
2. explain 변수 결측값 제거
3. menu, explain 중 '버거' 단어 포함되지 않은 값 제거
4. menu, explain 중 '세트' 단어 포함된 값 제거

In [ ]:
df = df.rename(columns={'ids': 'id'})

In [ ]:
df.head(100)

,Unnamed: 0,id,name,rank,visitor,blog,menu,explain,price,jibun_address
0,0,37282209,베이킹 도우,4.45,205,212,마르게리따,NaN,"16,500원",원서동 156
1,1,37282209,베이킹 도우,4.45,205,212,프로슈토 루꼴라,NaN,"24,500원",원서동 156
2,2,37282209,베이킹 도우,4.45,205,212,초리조,NaN,"18,500원",원서동 156
3,3,37282209,베이킹 도우,4.45,205,212,루꼴라,NaN,"19,500원",원서동 156
4,4,37282209,베이킹 도우,4.45,205,212,필스너 우르켈 생맥주,NaN,"8,000원",원서동 156
...,...,...,...,...,...,...,...,...,...,...
95,95,1966454804,버거스올마이티 강남역점,4.74,"1,767",731,치킨텐더(4조각),NaN,"5,000원",역삼동 813-4
96,96,1966454804,버거스올마이티 강남역점,4.74,"1,767",731,통새우너겟(2조각),NaN,"3,000원",역삼동 813-4
97,97,1966454804,버거스올마이티 강남역점,4.74,"1,767",731,통새우너겟(4조각),NaN,"5,000원",역삼동 813-4
98,98,1966454804,버거스올마이티 강남역점,4.74,"1,767",731,탄산음료(리필가능),"펩시, 칠성사이다, 마운틴듀,제로펩시","3,000원",역삼동 813-4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4732 entries, 0 to 4731
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  4732 non-null   int64  
 1   id          4732 non-null   int64  
 2   name        4732 non-null   object 
 3   rank        4732 non-null   float64
 4   visitor     4732 non-null   object 
 5   blog        4732 non-null   object 
 6   menu        4732 non-null   object 
 7   explain     2751 non-null   object 
 8   price       4732 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 332.8+ KB


In [ ]:
df.describe()

,Unnamed: 0,id,rank
count,4732.000000,4.732000e+03,4732.000000
mean,2365.500000,1.058513e+09,3.198719
std,1366.155067,6.932359e+08,2.049896
min,0.000000,1.172006e+07,0.000000
25%,1182.750000,3.869228e+07,0.000000
50%,2365.500000,1.244364e+09,4.440000
75%,3548.250000,1.641303e+09,4.560000
max,4731.000000,1.988186e+09,4.940000


In [ ]:
# 결측치 제거
df_wo_set_nan = df.dropna(subset=['explain'])

In [ ]:
# 버거가 아닌 메뉴 제거
df_filtered = df_wo_set_nan[(df_wo_set_nan['menu'].str.contains('버거|burger', na=False, case=False)) |
                     (df_wo_set_nan['explain'].str.contains('버거|burger', na=False, case=False))]

In [ ]:
# 세트 메뉴 제거
df_filtered = df_filtered[~df_filtered['explain'].str.contains('세트', na=False)]
df_filtered = df_filtered[~df_filtered['explain'].str.contains('set', na=False)]
df_filtered = df_filtered[~df_filtered['menu'].str.contains('세트', na=False)]
df_filtered = df_filtered[~df_filtered['menu'].str.contains('set', na=False)]

In [ ]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2858 entries, 9 to 11514
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2858 non-null   int64  
 1   id             2858 non-null   int64  
 2   name           2858 non-null   object 
 3   rank           2858 non-null   float64
 4   visitor        2858 non-null   object 
 5   blog           2858 non-null   object 
 6   menu           2858 non-null   object 
 7   explain        2858 non-null   object 
 8   price          2858 non-null   object 
 9   jibun_address  2752 non-null   object 
dtypes: float64(1), int64(2), object(7)
memory usage: 245.6+ KB


In [ ]:
df_filtered.to_csv('/content/drive/MyDrive/24-2 KUBIG reco/final_data/df_after_isnull.csv', index=False)

#### 2. GPT 로 Patty 열 만들기

GPT 4.0 을 이용해 explain 데이터를 사용해 patty 데이터 생성

[프롬프팅]

버거 메뉴에 대한 데이터 처리를 하려고 해.
내가 입력한 csv 파일에서 'explain' 열을 읽고, 각 행마다 똑같은 작업을 수행해줘.  
수행 내용은 다음과 같아

1. explain 내용에 따라 패티 종류를 구분한다. 패티의 종류는 다음과 같다.  

'소고기 패티'
'치킨 패티'
'새우 패티'  

2. 판단이 어려운 explain 에 경우 '소고기 패티'로 처리한다.

3. 구분한 패티 종류를 'patty' 라는 새로운 열에 입력한다.

예를들어 볼게.  explain이 ' '소고기120g,아메리칸체다치즈,해쉬브라운,베이컨,스파이시소스를 곁들인 치즈버거' 인 경우는 patty가 '소고기 패티'  explain '크리스피함을 살리고 적양파로 상큼함을 주는 치킨보이 버거'인 경우는 patty가 '치킨 패티'야

모든 작업을 마치고 csv 형태로 출력해줘

#### 3. 메뉴 유사도 비교를 위한 형태소 추출

In [ ]:
!pip install konlpy
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

### 자연어 처리 실험

1. 정규표현식으로 한글, 숫자만 남기기

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/24-2 KUBIG reco/final_data/df_after_gpt.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2858 entries, 0 to 2857
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2858 non-null   int64  
 1   id             2858 non-null   int64  
 2   name           2858 non-null   object 
 3   rank           2858 non-null   float64
 4   visitor        2858 non-null   object 
 5   blog           2858 non-null   object 
 6   menu           2858 non-null   object 
 7   explain        2858 non-null   object 
 8   price          2858 non-null   object 
 9   jibun_address  2752 non-null   object 
 10  patty          2858 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 245.7+ KB


In [ ]:
def extract_hangul_and_numbers(text):
  hangul_and_numbers = re.sub('[^가-힣0-9]', ' ', text)
  return hangul_and_numbers

In [ ]:
example = extract_hangul_and_numbers(df['explain'][1])
print("전처리 이전: ",df['explain'][1])
print("전처리 이후: ",example)

전처리 이전:  파프리카 베이스의 핫오일과 새콤한 적채 피클을 곁들인 스파이시 치킨보이 버거
전처리 이후:  파프리카 베이스의 핫오일과 새콤한 적채 피클을 곁들인 스파이시 치킨보이 버거


2. 띄어쓰기 교정

In [ ]:
from pykospacing import Spacing
spacing = Spacing()

In [ ]:
def spacing_text(text):
  spaced_text = spacing(text)
  return spaced_text

In [ ]:
spaced = spacing_text(example)
print("전처리 이전: ",example)
print("전처리 이후: ",spaced)

전처리 이전:  파프리카 베이스의 핫오일과 새콤한 적채 피클을 곁들인 스파이시 치킨보이 버거
전처리 이후:  파프리카 베이스의 핫 오일과 새콤한 적채 피클을 곁들인 스파이시 치킨 보이 버거


3. 형태소 분석

In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
def extract_morphs(text):
  morphs_ = okt.morphs(text, stem=True)
  return morphs_

In [ ]:
morphs_ = extract_morphs(spaced)
print("전처리 이전: ",spaced)
print("전처리 이후: ",morphs_)

전처리 이전:  파프리카 베이스의 핫 오일과 새콤한 적채 피클을 곁들인 스파이시 치킨 보이 버거
전처리 이후:  ['파프리카', '베이스', '의', '핫', '오일', '과', '새', '콤', '한', '적채', '피클', '을', '곁들이다', '스파이', '시', '치킨', '보이', '버거']


4. 불용어 제거

In [ ]:
with open('/content/drive/MyDrive/24-2 KUBIG reco/stopword.txt') as f:
    list_file = f.readlines()

stopwords_list = []
for stopword in list_file:
  stopwords = re.sub('[\n]', '', stopword)
  stopwords_list.append(stopwords)

def remove_stopwords(text):
  remove_stop = [x for x in text if x not in stopwords_list]
  return remove_stop

In [ ]:
remove_stop = remove_stopwords(morphs_)
print("전처리 이전: ",morphs_)
print("전처리 이후: ",remove_stop)

전처리 이전:  ['파프리카', '베이스', '의', '핫', '오일', '과', '새', '콤', '한', '적채', '피클', '을', '곁들이다', '스파이', '시', '치킨', '보이', '버거']
전처리 이후:  ['파프리카', '베이스', '핫', '오일', '새', '콤', '한', '적채', '피클', '스파이', '시', '치킨', '보이']


### 통합

1. 불용어사전 단어추가   

[버거, 들어가다, 곁들이다, 만들다, 하다, 되다, 있다, 좋다, 많다, 그렇다]

2. except list 정리

[치즈 및 패티 종류 등]  

In [ ]:
from konlpy.tag import Okt
okt = Okt()

from pykospacing import Spacing
spacing = Spacing()


except_list = [
    '로메 치즈',
    '로메치즈',
    '모짜렐라 치즈',
    '모짜렐라치즈',
    '체다 치즈',
    '체다치즈',
    '소고기 패티',
    '소고기패티',
    '돼지고기 패티'
    '돼지고기패티',
    '닭가슴살',
    '닭다리살',
    '닭가슴살 패티',
    '닭다리살 패티'

    ]

with open('/content/drive/MyDrive/24-2 KUBIG reco/stopword.txt') as f:
    list_file = f.readlines()

stopwords_list = []
for stopword in list_file:
  stopwords = re.sub('[\n]', '', stopword)
  stopwords_list.append(stopwords)

def review_to_words(raw_review):
  text = re.sub('[^가-힣]', ' ', raw_review)
  text = spacing(text)
  text = okt.morphs(text, stem=True)
  text = [x for x in text if x not in stopwords_list]
  text = [x for x in text if len(x)>1 or x in except_list]
  text = " ".join(text)
  return text

In [ ]:
df['wordlist'] = df['explain'].apply(review_to_words)

#### 4. 주소지를 통한 위경도 추출

In [ ]:
pip install geopy

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
'''
# 결측치 제거
df = df.dropna(subset=['jibun_address'], how='all')
df = df.dropna(how='all')
df.info()
'''

<class 'pandas.core.frame.DataFrame'>
Index: 2746 entries, 0 to 2857
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2746 non-null   int64  
 1   id             2746 non-null   int64  
 2   name           2746 non-null   object 
 3   rank           2746 non-null   float64
 4   visitor        2746 non-null   object 
 5   blog           2746 non-null   object 
 6   menu           2746 non-null   object 
 7   explain        2746 non-null   object 
 8   price          2746 non-null   object 
 9   jibun_address  2746 non-null   object 
 10  patty          2746 non-null   object 
 11  wordlist       2746 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 343.4+ KB


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2858 entries, 0 to 2857
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2858 non-null   int64  
 1   id             2858 non-null   int64  
 2   name           2858 non-null   object 
 3   rank           2858 non-null   float64
 4   visitor        2858 non-null   object 
 5   blog           2858 non-null   object 
 6   menu           2858 non-null   object 
 7   explain        2858 non-null   object 
 8   price          2858 non-null   object 
 9   jibun_address  2752 non-null   object 
 10  patty          2858 non-null   object 
 11  wordlist       2858 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 268.1+ KB


카카오 맵 API를 통한 위경도 변환

In [ ]:
import requests, json

def get_location(address):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
  headers = {"Authorization": "KakaoAK c550c27b304a1c785fd432b138514c45"}
  api_json = json.loads(str(requests.get(url,headers=headers).text))
  address = api_json['documents'][0]['address']
  x_y = [float(address['y']), float(address['x'])]
  #crd = {"lat": str(address['y']), "lng": str(address['x'])}

  return x_y

In [ ]:
latitudeSet = []
longtitudeSet = []
failed_address = []

locations = df['jibun_address'].astype(str)

for idx, location in enumerate(locations):
    try:
        #print(get_location)
        x_y = get_location(location)
        latitudeSet.append(x_y[0])
        longtitudeSet.append(x_y[1])
    except:
        #print('주솟값 변환 실패: 직접 변경 필요')
        failed_address.append(idx)
        pass


불량 주소지 값 추출 및 제거

In [ ]:
failed_addresses = df.iloc[failed_address]['jibun_address'].tolist()
for address in failed_addresses:
    print(address)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
구갈동 227-32
구갈동 227-32
구갈동 227-32
구갈동 227-32
구갈동 227-32
구갈동 227-32
풍무동 277-9
영천동 358-37
영천동 358-37
영천동 358-37
우
우편번호
06506
우
우편번호
06506
우
우편번호
06506
우
우편번호
06506
우
우편번호
06506
방서동 271-2
방서동 271-2
방서동 271-2
방서동 271-2
우
우편번호
35369
우
우편번호
35369
우
우편번호
35369
우
우편번호
35369
우
우편번호
35369
우
우편번호
35369


In [ ]:
df = df.drop(failed_address)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2727 entries, 0 to 2857
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2727 non-null   int64  
 1   id             2727 non-null   int64  
 2   name           2727 non-null   object 
 3   rank           2727 non-null   float64
 4   visitor        2727 non-null   object 
 5   blog           2727 non-null   object 
 6   menu           2727 non-null   object 
 7   explain        2727 non-null   object 
 8   price          2727 non-null   object 
 9   jibun_address  2727 non-null   object 
 10  patty          2727 non-null   object 
 11  wordlist       2727 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 277.0+ KB


In [ ]:
df['latitude'] = latitudeSet
df['longitude'] = longtitudeSet

###5. WordList 내 중복 단어 제거

In [ ]:
# 중복 단어 제거 함수 정의
def remove_duplicates(wordlist):
    words = wordlist.split()
    seen = set()
    unique_words = []
    for word in words:
        if word not in seen:
            unique_words.append(word)
            seen.add(word)
    return ' '.join(unique_words)

# 'wordlist' 열에 함수 적용
df['wordlist'] = df['wordlist'].apply(remove_duplicates)

# 결과 출력
df

,Unnamed: 0,id,name,rank,visitor,blog,menu,explain,price,jibun_address,patty,wordlist,latitude,longitude
0,9,1263574785,버거보이 성수낙낙점,4.43,"1,049",606,빅보이 버거,"소고기 패티와 통살 새우와의 만남. 버거보이 특제소스, 토마토, 치커리가 들어간 빅...","10,500원",성수동2가 280,소고기 패티,소고기 패티 통살 새우 와의 보이 특제 소스 토마토 커리,37.546672,127.066125
1,10,1263574785,버거보이 성수낙낙점,4.43,"1,049",606,불고기 보이 버거,"깔끔한 머스타드 마요와 진한 불고기풍 소스, 아삭한 양상추가 조화로운 불고기 보이 버거","7,700원",성수동2가 280,소고기 패티,깔끔하다 머스타드 마요 진하다 불고기 소스 양상추 조화롭다 보이,37.546672,127.066125
2,11,1263574785,버거보이 성수낙낙점,4.43,"1,049",606,핫 치킨보이 버거,파프리카 베이스의 핫오일과 새콤한 적채 피클을 곁들인 스파이시 치킨보이 버거,"8,300원",성수동2가 280,치킨 패티,파프리카 베이스 오일 적채 피클 스파이 치킨 보이,37.546672,127.066125
3,12,1263574785,버거보이 성수낙낙점,4.43,"1,049",606,슈퍼 베지보이 버거,코리안 스타일 치폴레 소스와 신선한 루꼴라와 적양파가 들어간 슈퍼 베지보이 버거,"9,700원",성수동2가 280,소고기 패티,코리안 스타일 치폴레 소스 신선하다 루꼴라 양파 슈퍼 베다 보이,37.546672,127.066125
4,13,1263574785,버거보이 성수낙낙점,4.43,"1,049",606,치킨보이 버거,크리스피함을 살리고 적양파로 상큼함을 주는 치킨보이 버거,"7,900원",성수동2가 280,치킨 패티,크리스 피함 살리다 양파 상큼 주다 치킨 보이,37.546672,127.066125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2853,11510,1900636788,리미티드버거,4.91,237,173,아보카도 랜치버거,"100%소고기 패티에 아메리칸치즈,베이컨, 아보카도 슬라이스, 렌치소스 가 들어간 ...","9,700원",상암동 2-77,소고기 패티,소고기 패티 아메리칸 치즈 베이컨 아보카도 라이스 렌치 소스 가다 수제,37.578085,126.895868
2854,11511,1900636788,리미티드버거,4.91,237,173,하와이안버거,"100%소고기패티,청상추,토마토,아메리칸치즈,베이컨2장, 구운파인애플,바베큐소스,하...","9,700원",상암동 2-77,소고기 패티,소고기 패티 상추 토마토 아메리칸 치즈 베이컨 파인애플 바베큐 소스 하우스,37.578085,126.895868
2855,11512,1916776752,미국식 제주점,0.00,202,171,BUST BURGER,미국식 제주점의 시그니처 메뉴 토시살을 숙성 시켜 수비드 한 스테이크버거,"12,900원",노형동 1271-7,소고기 패티,미국 주점 메뉴 토시 숙성 비드 스테이크,33.490015,126.480019
2856,11513,1916776752,미국식 제주점,0.00,202,171,DOUBLE BEEF (커스텀메뉴),기존 버스트버거에 토시살 120g이 추가되는 커스텀 메뉴입니다.,"9,900원",노형동 1271-7,소고기 패티,기존 버스 토시 추가 크다 스텀 메뉴,33.490015,126.480019


In [ ]:
df.to_csv('/content/drive/MyDrive/24-2 KUBIG reco/final_data/df_premium_0603.csv', index=False)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2727 entries, 0 to 2857
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2727 non-null   int64  
 1   id             2727 non-null   int64  
 2   name           2727 non-null   object 
 3   rank           2727 non-null   float64
 4   visitor        2727 non-null   object 
 5   blog           2727 non-null   object 
 6   menu           2727 non-null   object 
 7   explain        2727 non-null   object 
 8   price          2727 non-null   object 
 9   jibun_address  2727 non-null   object 
 10  patty          2727 non-null   object 
 11  wordlist       2727 non-null   object 
 12  latitude       2727 non-null   float64
 13  longitude      2727 non-null   float64
dtypes: float64(3), int64(2), object(9)
memory usage: 319.6+ KB


### 프랜차이즈 버거

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/24-2 KUBIG reco/final_data/data_4.csv')

##### 1. 데이터 전처리

1. 데이터 변수명 수정
2. ingredient, explain 열 통합

In [ ]:
df.head()

,작성자 이름,브랜드,버거 이름,가격,한줄 설명,"재료(콤마(,)로 구분하기)"
0,최주희,롯데리아,왕돈까스버거,7500,압도적인 크기의 K-style 왕돈까스버거,"돈까스, 저민 양상추, 저민 피클, 마요네즈 소스, 케첩 소스, 데리야끼 소스"
1,최주희,롯데리아,매운 왕돈까스버거,7500,압도적인 크기의 매콤한 왕돈까스버거,"돈까스, 저민 양상추, 저민 피클, 마요네즈 소스, 케첩 소스, 데리야끼 소스"
2,최주희,롯데리아,전주 비빔라이스 버거,6900,"전주비빔밥을 담은 라이스번과 노른자가 터지는 반숙란,달콤한 고추장 소스가 어우러진 ...","라이스번, 반숙란, 양파, 양상추, 고추장소스, 소고기패티"
3,최주희,롯데리아,새우베이컨버거,5800,베이컨+토마토로 더욱 풍성해진 맛과 볼륨감의 새우버거 업그레이드 한정판,"타르타르소스, 새우패티, 양상추, 사우전드 아일랜드 드레싱, 베이컨, 토마토"
4,최주희,롯데리아,불고기 베이컨,5800,베이컨+토마토로 더욱 풍성해진 맛과 볼륨감의 불고기버거 업그레이드 한정판,"마요네즈, 양파, 불고기패티, 불고기소스, 상추, 베이컨, 토마토"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   작성자 이름           111 non-null    object
 1   브랜드              111 non-null    object
 2   버거 이름            111 non-null    object
 3   가격               111 non-null    int64 
 4   한줄 설명            110 non-null    object
 5   재료(콤마(,)로 구분하기)  111 non-null    object
dtypes: int64(1), object(5)
memory usage: 5.3+ KB


In [ ]:
df = df.drop(columns=['작성자 이름'])

In [ ]:
df.rename(columns={
    '브랜드': 'brand',
    '버거 이름': 'name',
    '가격': 'price',
    '한줄 설명': 'explain',
    '재료(콤마(,)로 구분하기)': 'ingredient'
}, inplace=True)


In [ ]:
# 'explain' 열과 'ingredient' 열을 결합하여 'explain' 열에 다시 저장
df['explain'] = df['explain'] + ' ' + df['ingredient']

# 'ingredient' 열 삭제
df.drop(columns=['ingredient'], inplace=True)

In [ ]:
df

,brand,name,price,explain
0,롯데리아,왕돈까스버거,7500,"압도적인 크기의 K-style 왕돈까스버거 돈까스, 저민 양상추, 저민 피클, 마요..."
1,롯데리아,매운 왕돈까스버거,7500,"압도적인 크기의 매콤한 왕돈까스버거 돈까스, 저민 양상추, 저민 피클, 마요네즈 소..."
2,롯데리아,전주 비빔라이스 버거,6900,"전주비빔밥을 담은 라이스번과 노른자가 터지는 반숙란,달콤한 고추장 소스가 어우러진 ..."
3,롯데리아,새우베이컨버거,5800,베이컨+토마토로 더욱 풍성해진 맛과 볼륨감의 새우버거 업그레이드 한정판 타르타르소스...
4,롯데리아,불고기 베이컨,5800,"베이컨+토마토로 더욱 풍성해진 맛과 볼륨감의 불고기버거 업그레이드 한정판 마요네즈,..."
...,...,...,...,...
106,맘스터치,통새우버거,3500,"고소하고 탱글한 통새우 패티에 신선한 양상추가 조화로운 버거. 통새우 패티, 맘스소..."
107,맘스터치,화이트갈릭버거,4900,부드러운 화이트 갈릭소스에 프리미엄 더블햄과 통가슴살 패티까지 담은 묵직한 버거. ...
108,맘스터치,디럭스불고기버거,5200,"진한 불고기 소스의 패티가 두장, 고소한 치즈와 신선한 양상추가 곁들어진 프리미엄 ..."
109,맘스터치,할라피뇨통살버거,4800,바삭한 통다리살 패티와 매콤한 할라피뇨 소스가 어우러져 뒷맛까지 깔끔한 버거. 닭넓...


In [ ]:
df.to_csv('/content/drive/MyDrive/24-2 KUBIG reco/final_data/df_f_after_isnull.csv', index=False)

#### 2. GPT 로 Patty 열 만들기

[프롬프팅]

버거 메뉴에 대한 데이터 처리를 하려고 해.
내가 입력한 csv 파일에서 'explain' 열을 읽고, 각 행마다 똑같은 작업을 수행해줘.  
수행 내용은 다음과 같아

1. explain 내용에 따라 패티 종류를 구분한다. 패티의 종류는 다음과 같다.  

'소고기 패티'
'치킨 패티'
'새우 패티'  

2. 판단이 어려운 explain 에 경우 '소고기 패티'로 처리한다.

3. 구분한 패티 종류를 'patty' 라는 새로운 열에 입력한다.

예를들어 볼게.  explain이 ' '소고기120g,아메리칸체다치즈,해쉬브라운,베이컨,스파이시소스를 곁들인 치즈버거' 인 경우는 patty가 '소고기 패티'  explain '크리스피함을 살리고 적양파로 상큼함을 주는 치킨보이 버거'인 경우는 patty가 '치킨 패티'야

모든 작업을 마치고 csv 형태로 출력해줘

#### 3. 메뉴 유사도 비교를 위한 형태소 추출




In [ ]:
df = pd.read_csv('/content/drive/MyDrive/24-2 KUBIG reco/final_data/df_f_after_gpt.csv')

In [ ]:
df['explain'] = df['explain'].astype(str)
df['wordlist'] = df['explain'].apply(review_to_words)

In [ ]:
# 'wordlist' 열에 함수 적용
df['wordlist'] = df['wordlist'].apply(remove_duplicates)
df

,brand,name,price,explain,patty,wordlist
0,롯데리아,왕돈까스버거,7500,"압도적인 크기의 K-style 왕돈까스버거 돈까스, 저민 양상추, 저민 피클, 마요...",소고기 패티,압도 크기 돈까스 양상추 피클 마요네즈 소스 케첩 데리야끼
1,롯데리아,매운 왕돈까스버거,7500,"압도적인 크기의 매콤한 왕돈까스버거 돈까스, 저민 양상추, 저민 피클, 마요네즈 소...",소고기 패티,압도 크기 매콤 왕돈 끄다 돈까스 양상추 피클 마요네즈 소스 케첩 데리야끼
2,롯데리아,전주 비빔라이스 버거,6900,"전주비빔밥을 담은 라이스번과 노른자가 터지는 반숙란,달콤한 고추장 소스가 어우러진 ...",소고기 패티,비빔밥 담다 라이스 노른자 터지다 숙란 달콤하다 고추장 소스 어우러지다 새롭다 양파...
3,롯데리아,새우베이컨버거,5800,베이컨+토마토로 더욱 풍성해진 맛과 볼륨감의 새우버거 업그레이드 한정판 타르타르소스...,새우 패티,베이컨 토마토 더욱 성하다 볼륨 새우 업그레이드 한정판 타르 소스 패티 양상추 우전...
4,롯데리아,불고기 베이컨,5800,"베이컨+토마토로 더욱 풍성해진 맛과 볼륨감의 불고기버거 업그레이드 한정판 마요네즈,...",소고기 패티,베이컨 토마토 더욱 성하다 볼륨 불고기 업그레이드 한정판 마요네즈 양파 패티 소스 상추
...,...,...,...,...,...,...
106,맘스터치,통새우버거,3500,"고소하고 탱글한 통새우 패티에 신선한 양상추가 조화로운 버거. 통새우 패티, 맘스소...",새우 패티,고소하다 새우 패티 신선하다 양상 추가 조화롭다 맘스 소스 양상추 양파 피클
107,맘스터치,화이트갈릭버거,4900,부드러운 화이트 갈릭소스에 프리미엄 더블햄과 통가슴살 패티까지 담은 묵직한 버거. ...,소고기 패티,부드럽다 화이트 갈릭 소스 프리미엄 더블 가슴 패티 담다 직하다 치즈 양파 피클
108,맘스터치,디럭스불고기버거,5200,"진한 불고기 소스의 패티가 두장, 고소한 치즈와 신선한 양상추가 곁들어진 프리미엄 ...",소고기 패티,진하다 불고기 소스 패티 고소하다 치즈 신선하다 양상 추가 들다 프리미엄 혼합 양상...
109,맘스터치,할라피뇨통살버거,4800,바삭한 통다리살 패티와 매콤한 할라피뇨 소스가 어우러져 뒷맛까지 깔끔한 버거. 닭넓...,소고기 패티,바삭 다리 패티 매콤 피뇨 소스 어우러지다 뒷맛 깔끔하다 넓적다리 랠리 양상추 양파...


In [ ]:
df.to_csv('/content/drive/MyDrive/24-2 KUBIG reco/final_data/df_franchise.csv', index=False)

## 3. 콘텐츠 기반 추천 시스템

In [ ]:
import pandas as pd
pip install chardet
franchise_burger = pd.read_csv('/content/drive/MyDrive/쿠빅 컨퍼런스(추천시스템)/df_franchise (1).csv')
#인코딩 안되서 추가로 넣은 코드. 로컬에서 바로 데이터 다운받을수 있다고 하면 read.csv만 해도 됨!

import chardet

with open('/content/drive/MyDrive/쿠빅 컨퍼런스(추천시스템)/df_premium.csv', 'rb') as file:
    raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding_premium = result['encoding']

premium_burger = pd.read_csv('/content/drive/MyDrive/쿠빅 컨퍼런스(추천시스템)/df_premium.csv',  encoding=encoding_premium )
franchise_burger.head()
premium_burger.head()
# 프리미엄 버거 가격 숫자형 데이터로 바꾸는 작업


In [ ]:

# '원' 지우고 진행
premium_burger['price'] = premium_burger['price'].replace({'원': '', ',': ''}, regex=True)

# 숫자형으로 변환할 수 없는 경우 nan 처리
premium_burger['price'] = pd.to_numeric(premium_burger['price'], errors='coerce')

# price에서 결측값 있는 행 제거
premium_burger = premium_burger.dropna(subset=['price'])

# 정수형으로 바꾸기
premium_burger['price'] = premium_burger['price'].astype(int)
franchise_burger = franchise_burger.rename(columns={
    'brand': 'name',
    'name': 'menu',
})


# 프랜차이즈에서 관련있는 행만 모으기
filtered_franchise_burger = franchise_burger[[ 'name', 'menu', 'price', 'wordlist', 'patty']]
# 프랜차이즈, 프리미엄 클래스 구분

filtered_franchise_burger['class'] = 0
premium_burger['class'] = 1
burger_data = pd.concat([filtered_franchise_burger, premium_burger], ignore_index = True)

burger_data.tail()
burger_data['visitor'] = burger_data['visitor'].fillna('0')
burger_data['blog'] = burger_data['blog'].fillna('0')

burger_data['visitor'] = burger_data['visitor'].str.replace(',', '').astype(int)
burger_data['blog'] = burger_data['blog'].str.replace(',', '').astype(int)


In [ ]:

max_visitors = burger_data['visitor'].max()
max_blogs = burger_data['blog'].max()
burger_data['wordlist'] = burger_data['wordlist'].astype(str)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:

tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(' '))
tfidf_matrix = tfidf_vectorizer.fit_transform(burger_data['wordlist'])


In [ ]:

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [ ]:

# 코사인 유사도를 DataFrame으로 변환하여 확인
similarity_df = pd.DataFrame(cosine_sim, index=burger_data['menu'], columns=burger_data['menu'])
similarity_df.head()
selected_burger = '통새우와퍼'

In [ ]:
def final_recommendation(burger_data, selected_burger, min, max, popularity_min):

  selected_burger_patty = burger_data[burger_data['menu'] == selected_burger]['patty'].values[0]

  final_scores = similarity_df[selected_burger].values

  recommendations_df = pd.DataFrame({
      'id': burger_data['id'],
    'menu': burger_data['menu'],
    'name': burger_data['name'],
    'class': burger_data['class'],
    'price': burger_data['price'],
    'patty': burger_data['patty'],
    'visitor': burger_data['visitor'],
    'blog': burger_data['blog'],
    'score': final_scores
  })

  filtered_recommendations = recommendations_df[
        (recommendations_df['class'] == 1) &
        (recommendations_df['price'] >= min) &
        (recommendations_df['price'] < max) &
        ((recommendations_df['visitor'] + recommendations_df['blog']) >= popularity_min) &
        (recommendations_df['patty'].str.contains(selected_burger_patty, na = False))
  ]

  filtered_recommendations = filtered_recommendations.drop_duplicates(subset='menu')

  # 최종 추천 점수를 계산하여 상위 5개 추천 확인
  final_recommendations = filtered_recommendations[['id', 'menu', 'name', 'price', 'score']].set_index('menu').sort_values(by='score', ascending=False).iloc[1:11]
  return final_recommendations

In [ ]:
final_recommendation(burger_data, selected_burger, 5000, 15000, 0)

## 4. ALS 협업 필터링 기반 추천 시스템

In [ ]:
!pip install implicit
import numpy as np
# from tqdm import tqdm_noterest as tqdm
import tqdm
from implicit.evaluation import *  
from implicit.als import AlternatingLeastSquares as ALS  
# from implicit.bpr import BayesianPersonalizedRanking as BPR  


In [ ]:
!set OPENBLAS_NUM_THREADS=1
# ALS 모델 초기화
als_model = ALS(factors=20, regularization=0.01, iterations=100)
# threadpoolctl.threadpool_limits(1, "blas")
# factors: 잠재 요인의 수
# regularization: 정규화 파라미터
# iterations: 반복 횟수
import pandas as pd


In [ ]:

train = pd.read_csv('1차 리뷰 전처리(원본).csv')
train = train[['username','restaurant']]
train.columns = ['user_id', 'rest_id']
train.head()
user2idx = {}
for i, l in enumerate(train['user_id'].unique()):
    user2idx[l] = i
    
rest2idx = {}
for i, l in enumerate(train['rest_id'].unique()):
    rest2idx[l] = i

idx2user = {}
for i, l in enumerate(train['user_id'].unique()):
    idx2user[i] = l

idx2rest = {}
for i, l in enumerate(train['rest_id'].unique()):
    idx2rest[i] = l
# train : 리뷰크롤링 데이터로 만듦
data = train[['user_id', 'rest_id']].reset_index(drop=True)
useridx = data['useridx'] = train['user_id'].apply(lambda x: user2idx[x]).values
restidx = data['restidx'] = train['rest_id'].apply(lambda x: rest2idx[x]).values
rating = np.ones(len(data))
data.head()
len(rating)
import scipy 

purchase_sparse = scipy.sparse.csr_matrix((rating, (useridx, restidx)), shape=(len(set(useridx)), len(set(restidx))))
purchase_sparse
# ALS 모델 학습
# purchase_sparse.T: 사용자-아이템 행렬의 전치 행렬 (얘만 구해서 넣으면 됨)
als_model.fit(purchase_sparse)
# 사용자 0에 대한 상위 200개의 추천 생성, 첫 10개 항목 확인
als_model.recommend(0, purchase_sparse[0], N=100)[0:10]

visit_list = train.groupby(['user_id'])['rest_id'].agg({'unique'}).reset_index()
visit_list.head()

In [ ]:
def recommend(userids, visit_list, idx2user, purchase_sparse, idx2rest):
    total_rec_list = {}
    
    for user in userids:
        rec_list = []  # 현재 사용자의 추천 리스트 초기화
        
        # 사용자가 이미 방문한 가게의 ID를 'seen'에 저장
        seen = visit_list[visit_list['user_id'] == idx2user[user]]['unique'].values[0]
        
        # ALS 모델을 사용하여 현재 사용자에 대한 상위 100개의 추천 생성
        recs = als_model.recommend(user, purchase_sparse[user], N=100)[0]
        # print(recs)
        
        # 추천된 가게 인덱스를 가게 ID로 변환하고 상위 10개 선택
        recs = [idx2rest[x] for x in recs][0:10]
        
        # 이미 방문한 가게를 제외한 추천 리스트 생성
        for rec in recs:
            if rec not in seen:
                rec_list.append(rec)
        
        # 추천 리스트가 50개 미만인 경우, 유명 가게로 보충
        if len(rec_list) < 10:
            for i in popular_rec_model:
                if i not in seen:  # 보충 추천 목록에 이미 방문한 가게가 포함되지 않도록 함
                    rec_list.append(i)
                if len(rec_list) == 10 : break
        
        # 현재 사용자의 추천 리스트를 딕셔너리에 저장 (최대 100개)
        total_rec_list[idx2user[user]] = rec_list
        return total_rec_list

In [ ]:
users = [0, 1, 2]
rec_list = recommend(users, visit_list, idx2user, purchase_sparse, idx2rest)
rec_list[idx2user[0]]

[ 84 213 366 237 302 109 382 181 358 197 295 128 308 272 246 266 260 334
 119 135 369 327 130 155 143  29  56 195 168 102 319 169 125  47 142 144
 184 140 204  53  92  18 139 252  37 326 147 287 124 281 152 273  28 215
   6 150  46   3 105  63 220 294  24 199  93 118  19   2 253 208 233 256
 101 242 179 200 368 222 170 286 353 243  58 362 194 299 221 284 278 225
  48 376 317 337 371 132  41  16 151  54]


['엘더버거 성수',
 '판포리아',
 '라모스버거',
 '안녕육지사람',
 'bd버거 성수',
 '사천 010',
 '릿잇타미',
 '번패티번 롯데백화점 부산본점',
 '헤이로라 망원동 본점',
 '리프버거']